<a href="https://colab.research.google.com/github/vblagoje/notebooks/blob/main/haystack2x-demos/haystack_rag_serperdev_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction

This notebook showcases the upcoming Haystack OpenAPI service-based Retriever-Augmented Generation (RAG). Given a user query, we search the web for the results of the query and inject these results into LLM contenxt along with the system prompt.

## 1. Setup

Let's install necessary libraries and import key modules to build the foundation for the subsequent steps.

In [1]:
!pip uninstall -y llmx

Found existing installation: llmx 0.0.15a0
Uninstalling llmx-0.0.15a0:
  Successfully uninstalled llmx-0.0.15a0


In [2]:
!pip install -q openapi3 jsonref

In [3]:
!pip install -q git+https://github.com/deepset-ai/haystack.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.1 MB/s eta 0:00:00


In [5]:
import getpass
import os
import json
import requests

from haystack import Pipeline
from haystack.components.generators.utils import default_streaming_callback
from haystack.components.converters import OpenAPIServiceToFunctions
from haystack.components.generators.chat import GPTChatGenerator
from haystack.components.connectors import OpenAPIServiceConnector
from haystack.dataclasses import ChatMessage

## 2. Prepare SerperDev service along with system prompt

- Generate OpenAI functions definitions for SerperDev

In [6]:
gen_func_pipeline = Pipeline()
gen_func_pipeline.add_component("spec_to_functions", OpenAPIServiceToFunctions())

In [8]:
functions_result = gen_func_pipeline.run(data={"sources":["https://bit.ly/3NIJqnd"],
                                               "system_messages":[requests.get("https://bit.ly/3TdHsyB").text]})

## 3. API keys, set up simple authentication mechanism

In [9]:
llm_api_key = getpass.getpass("Enter LLM provider api key:")


Enter LLM provider api key:··········


In [10]:
serper_dev_key = getpass.getpass("Enter serperdev api key:")
services_auth = {"SerperDev":serper_dev_key}

Enter serperdev api key:··········


## 4. Retrieval step - SerperDev service invocation

In [11]:
invoke_service_pipe = Pipeline()
invoke_service_pipe.add_component("functions_llm", GPTChatGenerator(api_key=llm_api_key, model_name="gpt-3.5-turbo-0613"))
invoke_service_pipe.add_component("openapi_container", OpenAPIServiceConnector(services_auth))
invoke_service_pipe.connect("functions_llm.replies", "openapi_container.messages")

In [23]:
user_prompt = "Why was Sam Altman ousted from OpenAI?"


In [33]:
service_desc_document = functions_result["spec_to_functions"]["documents"][0]
openai_functions_definition = json.loads(service_desc_document.content)
openapi_spec = service_desc_document.meta["spec"]

service_response = invoke_service_pipe.run(data={"messages":[ChatMessage.from_user(user_prompt)],
                                                 "generation_kwargs": {"functions": [openai_functions_definition]},
                                                 "service_openapi_spec": openapi_spec})

## 6. Generate LLM response

Inject service response into LLM context, pair it with system prompt

In [34]:
gen_pipe = Pipeline()
llm = GPTChatGenerator(api_key=llm_api_key, model_name="gpt-4-1106-preview", streaming_callback=default_streaming_callback)
gen_pipe.add_component("llm", llm)

github_pr_prompt_messages = [ChatMessage.from_system(service_desc_document.meta["system_message"])] + service_response["openapi_container"]["service_response"]
final_result = gen_pipe.run(data={"messages": github_pr_prompt_messages})

Sam Altman was ousted from OpenAI because, according to new reports, he may have been a manipulative leader. These reports suggest that he pitted board members and employees against each other to maintain power.